<a href="https://colab.research.google.com/github/onehungrybird/Agentic_AI_travel_planner/blob/main/memory_augmented_agents4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install faiss-cpu transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 19.9 MB/s eta 0:00:00


In [3]:
!pip install langchain_community
!pip install -U transformers accelerate bitsandbytes --quiet
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 59.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 59.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5

In [4]:
from google.colab import userdata
hf_token = userdata.get('HF_TOKEN')

In [5]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from transformers import pipeline

In [6]:
embedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

dimension = 384
index = faiss.IndexFlatL2(dimension)

memory_store = []

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
def add_memory(text):
  global memory_store, index
  embedding = embedding_model.encode([text])
  index.add(np.array(embedding, dtype=np.float32))
  memory_store.append(text)
  print(f"Added memory: {text}")

In [8]:
def retrieve_memory(query, top_k=3):
  query_embedding = embedding_model.encode([query])
  _, indices = index.search(np.array(query_embedding, dtype=np.float32), top_k)

  relevant_memories = [memory_store[i] for i in indices[0] if i < len(memory_store)]
  return relevant_memories

In [9]:
import torch
from langchain.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig

quant_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True
)

model_name = "mistralai/Mistral-7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config = quant_config,
    device_map = 'auto'
)

llm = pipeline("text-generation", model=model, tokenizer=tokenizer, device_map="auto")

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

Device set to use cuda:0


In [10]:
def agent_respond(query):
  relevant_memories = retrieve_memory(query)
  print("Relevant memories:===========", relevant_memories)
  context = " ".join(relevant_memories) if relevant_memories else "No prior memory found."

  prompt = f"Previous experiences: {context}\nCurrent query: {query}\nResponse:"
  response = llm(prompt, max_length=1000, do_sample=True, temperature=0.7)
  return response[0]['generated_text']

In [11]:
# Add some past experiences
add_memory("I previously answered a question about deep learning.")
add_memory("The user likes explanations with code examples.")

Added memory: I previously answered a question about deep learning.
Added memory: The user likes explanations with code examples.


In [12]:
# Query the agent
query = "Can you explain reinforcement learning?"
response = agent_respond(query)
print("Agent Response:", response)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Relevant memories:=========== ['I previously answered a question about deep learning.', 'The user likes explanations with code examples.', 'The user likes explanations with code examples.']
Agent Response: Previous experiences: I previously answered a question about deep learning. The user likes explanations with code examples. The user likes explanations with code examples.
Current query: Can you explain reinforcement learning?
Response: Sure, I can explain reinforcement learning with code examples. Reinforcement learning is a type of machine learning where an agent learns to make decisions by interacting with an environment. The environment provides feedback to the agent in the form of rewards or punishments, which help the agent learn how to make the best decisions.
Here's an example of a simple reinforcement learning problem:
Suppose you have a robot that can pick up objects from a table and move them to a target location. The environment is the table and the target location. The a